In [81]:
import pandas as pd
import numpy as np
import json

In [82]:
chunk_size = 5000000
index=0
df = pd.read_csv('./Project Data/ProjectTrainingData.csv', nrows=chunk_size, skiprows=index*chunk_size)

In [83]:
def do_sampling(dataframe):
    sample_rate=0.01
    if sample_rate < 1.0:
        np.random.seed(999)
        r1 = np.random.uniform(0, 1, dataframe.shape[0])
        dataframe = dataframe.iloc[r1 < sample_rate,:]
        print ("testing with small sample of training data, ", dataframe.shape)
        return dataframe

# Sampling to save time and parameter tuning. Dismiss this block in the last round 
df = do_sampling(df)

testing with small sample of training data,  (49905, 24)


In [84]:
df.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

df.describe()

,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,4.990500e+04,49905.000000,4.990500e+04,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000,49905.000000
mean,9.271719e+18,0.171285,1.410214e+07,1004.989540,0.258992,1.021120,0.221381,18522.858671,319.636269,59.059373,2082.732071,1.655445,190.664743,48857.658491,79.986995
std,5.315631e+18,0.376762,4.247125e+01,1.074629,0.471527,0.519493,0.686256,4681.933755,27.591077,46.422902,570.416769,1.362634,314.443947,50041.459266,63.310213
min,1.523000e+14,0.000000,1.410210e+07,1001.000000,0.000000,0.000000,0.000000,375.000000,216.000000,36.000000,112.000000,0.000000,33.000000,-1.000000,13.000000
25%,4.667723e+18,0.000000,1.410211e+07,1005.000000,0.000000,1.000000,0.000000,17163.000000,320.000000,50.000000,1873.000000,0.000000,35.000000,-1.000000,32.000000
50%,9.345456e+18,0.000000,1.410211e+07,1005.000000,0.000000,1.000000,0.000000,19998.000000,320.000000,50.000000,2260.000000,2.000000,39.000000,-1.000000,61.000000
75%,1.389682e+19,0.000000,1.410220e+07,1005.000000,1.000000,1.000000,0.000000,21684.000000,320.000000,50.000000,2496.000000,3.000000,169.000000,100084.000000,111.000000
max,1.844662e+19,1.000000,1.410221e+07,1012.000000,7.000000,5.000000,5.000000,21822.000000,1024.000000,1024.000000,2520.000000,3.000000,1835.000000,100248.000000,221.000000


In [85]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
37,1.000777e+19,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,e2a1ca37,2347f47a,...,1,0,15708,320,50,1722,0,35,-1,79
177,1.003023e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15705,320,50,1722,0,35,100084,79
217,1.003562e+19,0,14102100,1005,0,89a490f5,ce307e01,3e814130,ecad2386,7801e8d9,...,1,0,19771,320,50,2227,0,687,100075,48
333,1.005719e+18,0,14102100,1005,0,f282ab5a,61eb5bc4,f028772b,ecad2386,7801e8d9,...,1,0,20596,320,50,2161,0,35,-1,157
359,1.006040e+19,1,14102100,1005,0,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,...,1,0,17914,320,50,2043,2,39,-1,32


In [86]:
# check unique values in each column
for col in df.columns:
    print(col, len(df[col].value_counts()))

id 49905
click 2
hour 33
C1 7
banner_pos 6
site_id 1000
site_domain 842
site_category 18
app_id 1000
app_domain 76
app_category 22
device_id 9012
device_ip 40095
device_model 2558
device_type 4
device_conn_type 4
C14 571
C15 7
C16 8
C17 181
C18 4
C19 43
C20 155
C21 38


In [87]:
# check feature distribution
print(df.groupby('site_id')['click'].count().sort_values(ascending=False).head(10))
print(df.groupby('app_id')['click'].count().sort_values(ascending=False).head(10))

site_id
85f751fd    18315
1fbe01fe     8825
e151e245     2372
d9750ee7     1659
5b08c53b      999
856e6d3f      973
5b4d2eda      794
5ee41ff2      500
6399eda6      482
6256f5b4      391
Name: click, dtype: int64
app_id
ecad2386    31590
e2fcccd2     1467
7358e05e     1098
febd1138      953
66f5e02e      837
9c13b419      672
54c5d545      579
a5184c22      545
f0d41ff1      423
03528b27      379
Name: click, dtype: int64


In [88]:
# check out the proportion
print(f"top1 site id: {df[df['site_id'] == '85f751fd'].shape[0] / df.shape[0]}")
print(f"top1 app id: {df[df['app_id'] == 'ecad2386'].shape[0] / df.shape[0]}")

top1 site id: 0.36699729486023447
top1 app id: 0.6330027051397655


In [89]:
print(f" top1 site id count + top1 app id count: {df[df['site_id'] == '1fbe01fe'].shape[0] + df[df['app_id'] == 'ecad2386'].shape[0]}")
print(f" total dataset count: {df.shape[0]}")

# each data is either "site id == 1fbe01fe" or "app_id ==  ecad2386"

 top1 site id count + top1 app id count: 40415
 total dataset count: 49905


In [90]:


def feature_engineering(dataframe):
    
    dataframe = dataframe.drop(['id'], axis=1)
    
    # extract week & hour
    dataframe['Date'] = pd.to_datetime(dataframe['hour'], format='%y%m%d%H')
    dataframe['Day'] = dataframe['Date'].dt.day
    dataframe['Weekday'] = dataframe['Date'].dt.dayofweek
    dataframe['Hour'] = dataframe['Date'].dt.hour
    print(dataframe.head())
    dataframe = dataframe.drop(['Date', 'hour'], axis=1)

    # extract new feature from app id and site id
    dataframe.loc[:, 'is_ecad2386'] = 0
    dataframe.loc[dataframe.app_id=='ecad2386', 'is_ecad2386'] = 1
    print(dataframe.head())
    
    return dataframe
    

df2 = feature_engineering(df)

for col in df2.columns:
    print(col, len(df2[col].value_counts()))

     click      hour    C1  banner_pos   site_id site_domain site_category  \
37       0  14102100  1005           0  85f751fd    c4e18dd6      50e219e0   
177      0  14102100  1005           0  1fbe01fe    f3845767      28905ebd   
217      0  14102100  1005           0  89a490f5    ce307e01      3e814130   
333      0  14102100  1005           0  f282ab5a    61eb5bc4      f028772b   
359      1  14102100  1005           0  d9750ee7    98572c79      f028772b   

       app_id app_domain app_category  ... C16   C17 C18  C19     C20  C21  \
37   e2a1ca37   2347f47a     8ded1f7a  ...  50  1722   0   35      -1   79   
177  ecad2386   7801e8d9     07d7df22  ...  50  1722   0   35  100084   79   
217  ecad2386   7801e8d9     07d7df22  ...  50  2227   0  687  100075   48   
333  ecad2386   7801e8d9     07d7df22  ...  50  2161   0   35      -1  157   
359  ecad2386   7801e8d9     07d7df22  ...  50  2043   2   39      -1   32   

          Date  Day  Weekday  Hour  
37  2014-10-21   21      

In [91]:
from sklearn import preprocessing

def feature_encoding(dataframe):
    feats_to_encode = ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
                       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
                      'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'Day', 'Weekday', 'Hour', 'Weekday']
    for column in feats_to_encode:
        enc = preprocessing.LabelEncoder()
        dataframe[column] = enc.fit_transform(dataframe[column])
    
    print(dataframe.info())
    print(dataframe.head())
    
    return dataframe

df3 = feature_encoding(df2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49905 entries, 37 to 4999839
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   click             49905 non-null  int64
 1   C1                49905 non-null  int64
 2   banner_pos        49905 non-null  int64
 3   site_id           49905 non-null  int64
 4   site_domain       49905 non-null  int64
 5   site_category     49905 non-null  int64
 6   app_id            49905 non-null  int64
 7   app_domain        49905 non-null  int64
 8   app_category      49905 non-null  int64
 9   device_id         49905 non-null  int64
 10  device_ip         49905 non-null  int64
 11  device_model      49905 non-null  int64
 12  device_type       49905 non-null  int64
 13  device_conn_type  49905 non-null  int64
 14  C14               49905 non-null  int64
 15  C15               49905 non-null  int64
 16  C16               49905 non-null  int64
 17  C17               49905 non-

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing


def standardize_data(dataframe):
    scaler = preprocessing.StandardScaler().fit(dataframe)
    dataframe_scaled = scaler.transform(dataframe)
    return pd.DataFrame(dataframe_scaled, columns=dataframe.columns)

def split_data(dataframe):
    x_feats = dataframe.columns[dataframe.columns != 'click']
    X_train = dataframe[x_feats]
    y_train = dataframe['click']

    print(X_train.head())
    print(y_train.head())
    
    X_train = standardize_data(X_train)
    return X_train, y_train


X_train, y_train = split_data(df3)

     C1  banner_pos  site_id  site_domain  site_category  app_id  app_domain  \
37    2           0      525          658              5     889           9   
177   2           0      116          798              1     927          34   
217   2           0      540          685              3     927          34   
333   2           0      939          317             16     927          34   
359   2           0      841          505             16     927          34   

     app_category  device_id  device_ip  ...  C16  C17  C18  C19  C20  C21  \
37             12       2355      11015  ...    1   39    0    2    0   19   
177             0       5978      14317  ...    1   39    0    2   61   19   
217             0       5978      27835  ...    1   89    0   30   54   11   
333             0       5978       1505  ...    1   82    0    2    0   34   
359             0       5978      18520  ...    1   74    2    3    0    6   

     Day  Weekday  Hour  is_ecad2386  
37     0   

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing


def standardize_data(dataframe):
    scaler = preprocessing.StandardScaler().fit(dataframe)
    dataframe_scaled = scaler.transform(dataframe)
    return pd.DataFrame(dataframe_scaled, columns=dataframe.columns)

def split_data(dataframe):
    x_feats = dataframe.columns[dataframe.columns != 'click']
    X_train = dataframe[x_feats]
    y_train = dataframe['click']

    print(X_train.head())
    print(y_train.head())
    
    X_train = standardize_data(X_train)
    return X_train, y_train


X_train, y_train = split_data(df3)

     C1  banner_pos  site_id  site_domain  site_category  app_id  app_domain  \
37    2           0      525          658              5     889           9   
177   2           0      116          798              1     927          34   
217   2           0      540          685              3     927          34   
333   2           0      939          317             16     927          34   
359   2           0      841          505             16     927          34   

     app_category  device_id  device_ip  ...  C16  C17  C18  C19  C20  C21  \
37             12       2355      11015  ...    1   39    0    2    0   19   
177             0       5978      14317  ...    1   39    0    2   61   19   
217             0       5978      27835  ...    1   89    0   30   54   11   
333             0       5978       1505  ...    1   82    0    2    0   34   
359             0       5978      18520  ...    1   74    2    3    0    6   

     Day  Weekday  Hour  is_ecad2386  
37     0   

In [94]:
len(y_train)

49905

In [106]:
from sklearn.model_selection import train_test_split
col_name = df3.columns[df3.columns != 'click']
X = df3[col_name]
y = df3['click']

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3, random_state=12)


In [107]:
X_train.C1.size

34933

In [108]:
y_train.size

34933

In [109]:
X_train = X_train.drop(["site_id","app_id","device_id"],axis = 1 )

In [110]:
X_test = X_test.drop(["site_id","app_id","device_id"],axis = 1 )

In [115]:
rf = RandomForestClassifier() 

model = rf.fit(X_train, y_train)



In [116]:
predictions = model.predict_proba(X_test)
log_loss(y_test,predictions[:, 1])

0.5351536565479975

In [44]:
predictions

array([[0.938  , 0.062  ],
       [0.988  , 0.012  ],
       [0.962  , 0.038  ],
       ...,
       [0.936  , 0.064  ],
       [0.95   , 0.05   ],
       [0.98275, 0.01725]])

In [100]:
from sklearn.metrics import log_loss, make_scorer

def log_loss_scorer(y_true, y_pred):
    return log_loss(y_true, y_pred)

scorer = make_scorer(log_loss_scorer, greater_is_better = False, needs_proba=True)

In [119]:


def perform_rf(X_train, y_train):
    param_grid = {
    'n_estimators': [1000,1500,2000],
    'min_samples_leaf':[3,5,7]
     }  
   
    model = RandomForestClassifier(n_jobs = 3) 
    grid = GridSearchCV(model, param_grid=param_grid, cv=5, scoring=scorer, verbose=2 )
    grid.fit(X_train, y_train)

    
    
    print('Best score: ', -grid.best_score_)
    print('Best params: ', grid.best_params_)
    print(grid.cv_results_)
    return grid

best_grid = perform_rf(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] min_samples_leaf=3, n_estimators=1000 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ min_samples_leaf=3, n_estimators=1000, total=  23.9s
[CV] min_samples_leaf=3, n_estimators=1000 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.9s remaining:    0.0s


[CV] ............ min_samples_leaf=3, n_estimators=1000, total=  12.3s
[CV] min_samples_leaf=3, n_estimators=1000 ...........................
[CV] ............ min_samples_leaf=3, n_estimators=1000, total=  12.0s
[CV] min_samples_leaf=3, n_estimators=1000 ...........................
[CV] ............ min_samples_leaf=3, n_estimators=1000, total=  11.9s
[CV] min_samples_leaf=3, n_estimators=1000 ...........................
[CV] ............ min_samples_leaf=3, n_estimators=1000, total=  11.7s
[CV] min_samples_leaf=3, n_estimators=1500 ...........................
[CV] ............ min_samples_leaf=3, n_estimators=1500, total=  17.7s
[CV] min_samples_leaf=3, n_estimators=1500 ...........................
[CV] ............ min_samples_leaf=3, n_estimators=1500, total=  18.1s
[CV] min_samples_leaf=3, n_estimators=1500 ...........................
[CV] ............ min_samples_leaf=3, n_estimators=1500, total=  17.4s
[CV] min_samples_leaf=3, n_estimators=1500 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 12.3min finished


Best score:  0.4127029983320959
Best params:  {'min_samples_leaf': 7, 'n_estimators': 1000}
{'mean_fit_time': array([13.83702521, 16.84704061, 22.62975025, 10.21207123, 14.93455424,
       20.00451827,  9.71519651, 14.10858941, 18.47136207]), 'std_fit_time': array([4.72054841, 0.26004291, 0.11105603, 0.25577778, 0.10480499,
       0.51682006, 0.30581406, 0.43381295, 0.12567152]), 'mean_score_time': array([0.53158846, 0.81326146, 1.05595179, 0.51005311, 0.73117604,
       0.97282567, 0.53099308, 0.71226172, 0.91292133]), 'std_score_time': array([0.04513416, 0.00221765, 0.051083  , 0.00186451, 0.04268574,
       0.04923704, 0.03927473, 0.00256318, 0.00172881]), 'param_min_samples_leaf': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7],
             mask=[False, False, False, False, False, False, False, False,
                   False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[1000, 1500, 2000, 1000, 1500, 2000, 1000, 1500, 2000],
           

In [120]:
# train model with best parameters to get the result on vvalidation set

best_grid = RandomForestClassifier(n_jobs = 3, n_estimators = 1000, min_samples_leaf = 7) 
fit_rf = best_grid.fit(X_train, y_train)

In [121]:
predictions = fit_rf.predict_proba(X_test)
log_loss(y_test,predictions[:, 1])

0.4201734257098534